In [60]:
import os
import pandas as pd
from PIL import Image
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import classification_report

In [70]:
IMAGE_DIR = "C:\\Users\\deeks\\OneDrive\\Desktop\\project\\images"
LABEL_CSV = "C:\\Users\deeks\\OneDrive\\Desktop\\project\\memes.csv"
BATCH_SIZE = 16
EPOCHS = 10
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\deeks\AppData\Local\Temp\ipykernel_17956\3004790783.py:2: SyntaxWarning: invalid escape sequence '\d'
  LABEL_CSV = "C:\\Users\deeks\\OneDrive\\Desktop\\project\\memes.csv"


In [71]:
from torch.utils.data import Dataset
import pandas as pd
from PIL import Image
import os

class MemeDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, row["image"])  # <-- Key line
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = int(row["label"])
        return image, label

 

In [72]:
transform = transforms.Compose([
transforms.Resize((224, 224)),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225])
])

In [73]:
dataset = MemeDataset(LABEL_CSV, IMAGE_DIR, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [78]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2) # binary classification
model = model.to(DEVICE)

c:\Users\deeks\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\deeks\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [80]:
import pandas as pd
from collections import Counter
import torch

df = pd.read_csv( "C:\\Users\deeks\\OneDrive\\Desktop\\project\\memes.csv")
counts = Counter(df["label"])
print(counts)  # e.g., Counter({0: 110, 1: 30})


Counter({0: 91, 1: 49})


<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\deeks\AppData\Local\Temp\ipykernel_17956\1976548434.py:5: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv( "C:\\Users\deeks\\OneDrive\\Desktop\\project\\memes.csv")


In [81]:
total = sum(counts.values())
weight_0 = total / counts[0]
weight_1 = total / counts[1]

class_weights = torch.tensor([weight_0, weight_1], dtype=torch.float).to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=class_weights)


In [75]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [82]:
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader.dataset):.4f}")


Epoch 1, Loss: 0.7412
Epoch 2, Loss: 0.7328
Epoch 3, Loss: 0.7563
Epoch 4, Loss: 0.7549
Epoch 5, Loss: 0.7487
Epoch 6, Loss: 0.7383
Epoch 7, Loss: 0.7542
Epoch 8, Loss: 0.7475
Epoch 9, Loss: 0.7440
Epoch 10, Loss: 0.7582


In [83]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Classification Report:")
print(classification_report(all_labels, all_preds))


Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.37      0.47        19
           1       0.29      0.56      0.38         9

    accuracy                           0.43        28
   macro avg       0.47      0.46      0.43        28
weighted avg       0.53      0.43      0.44        28

